In [20]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm


from PIL import Image, ImageDraw
import glob
import os
import matplotlib.pyplot as plt

import numpy as np
import scipy.misc as smp

import json
from datetime import datetime

from operator import add
import re

from sklearn.cluster import KMeans

In [2]:
# include functions 
%run phase02-mask-generation/SugarUtils.py

./tiles/tile-x7620-y7600.png
./geometries/geo-x7620-y7600.geojson
./mask/mask-x7620-y7600.png


In [11]:
# helper function

def get_mask_list(pixels):
    mask_list = []

    for x in range(0,512):
        for y in range(0,512):
            if pixels[y,x] == (0,0,0,255):
                mask_list.append((y,x))
    return mask_list

def get_timeseries_image_paths(tile_x, tile_y, band):
    path = f"./Phase02-DataDelivery/sugarcanetiles/{tile_x}-{tile_y}-{band}*.png"
    images = glob.glob(path)
    return images

# need to sort the band_img list
# sort the image path according to timeseries

def get_image_path_dict(band_img):
    image_path_dict = {}
    for bi in band_img:
        timestr = bi[-14:][:10]
        t = datetime.strptime(timestr, '%Y-%m-%d')
        image_path_dict[t] = bi
    return image_path_dict

# store sugacrane pixel to tci_list
def get_sugarcrane_region_from_tci(mask_list, band_pixels):
    tci_list = []

    for mask_ind in mask_list:
        y = mask_ind[0]
        x = mask_ind[1]

        tci_list.append(band_pixels[y,x])
    return tci_list

def get_tci_dict(image_path_dict, mask_list):
    tci_dict = {}
    cnt = 0
    for elem in sorted(image_path_dict.keys()):
    #     mask_list = 

        path = image_path_dict[elem]
        img = Image.open(path)
        band_pixels = img.load()

        tci_list = get_sugarcrane_region_from_tci(mask_list, band_pixels)
        tci_dict[cnt] = tci_list
        cnt += 1
    return tci_dict


def which_pixel_over_timeseries(i, tci_dict):
    red_list = []
    green_list = []
    blue_list = []
    for key, value in tci_dict.items():
        color = value[i]      # this is the ith pixel 
        red_list.append(color[0])
        green_list.append(color[1])
        blue_list.append(color[2])
    
    return {"red_list":red_list, "green_list":green_list,"blue_list":blue_list}

def which_pixel_over_timeseries_ndvi(i, dict1):
    res = []
    for key, value in dict1.items():
        num = value[i]      # this is the ith pixel 
        res.append(num)
    
    return res


# for identifying cloud
def is_white_pixel(r,g,b):
    if r >= 225 and g >= 225 and b >= 225:
        return True
    return False


# for identifying cloud
def is_blue_pixel(r,g,b):
    total = r + g + b
    if b > 180 and b / total > 0.35:
        return True
    return False

# this function needs more modifications !!!!!

def is_green_pixel(r,g,b):
    total = r + g + b
    if total == 0:
        return False
    if g / total > 0.33334:
        return True
    return False

In [5]:
mask_files = f"./Phase02-DataDelivery/masks/*.png"
mask_images = glob.glob(mask_files)

# get all the x, y positions of tiles from masks file name
xy_pair = []
for mask_img in mask_images:
    mask_list = re.findall(r'\d+', mask_img)
    xy_pair.append( (mask_list[1],mask_list[2]) )

In [15]:
# building all data dataframe 



# output csv for futher processing 

# 71 days in total 
# for each day, need to calculate total pixel number that are green - total sugacrane area 

tile_xy_dict = {}
for xy in tqdm( xy_pair ):
    # get mask png data
    tile_x, tile_y = xy[0], xy[1]
    mask_file_path = f"./Phase02-DataDelivery/masks/mask-x{tile_x}-y{tile_y}.png"
    img = Image.open(mask_file_path)
    pixels = img.load()
    
    mask_list = get_mask_list(pixels)

    # get tci_dict
    """
    band_img = get_timeseries_image_paths(tile_x, tile_y, "TCI")
    image_path_dict = get_image_path_dict(band_img)
    tci_dict = get_tci_dict(image_path_dict, mask_list)
    """
    
    # for ndvi analysis 
    band_img_RED = get_timeseries_image_paths(tile_x, tile_y, "B04")
    image_path_dict_RED = get_image_path_dict(band_img_RED)
    red_dict = get_tci_dict(image_path_dict_RED, mask_list)

    band_img_NIR = get_timeseries_image_paths(tile_x, tile_y, "B08")
    image_path_dict_NIR = get_image_path_dict(band_img_NIR)
    nir_dict = get_tci_dict(image_path_dict_NIR, mask_list)
    
    # get the rest of the band 
    band_img_BLUE = get_timeseries_image_paths(tile_x, tile_y, "B02")
    image_path_dict_BLUE = get_image_path_dict(band_img_BLUE)
    blue_dict = get_tci_dict(image_path_dict_BLUE, mask_list)
    
    band_img_GREEN = get_timeseries_image_paths(tile_x, tile_y, "B03")
    image_path_dict_GREEN = get_image_path_dict(band_img_GREEN)
    green_dict = get_tci_dict(image_path_dict_GREEN, mask_list)
    
    band_img_VRE1 = get_timeseries_image_paths(tile_x, tile_y, "B05")
    image_path_dict_VRE1 = get_image_path_dict(band_img_VRE1)
    vre1_dict = get_tci_dict(image_path_dict_VRE1, mask_list)
    
    band_img_VRE2 = get_timeseries_image_paths(tile_x, tile_y, "B06")
    image_path_dict_VRE2 = get_image_path_dict(band_img_VRE2)
    vre2_dict = get_tci_dict(image_path_dict_VRE2, mask_list)
    
    band_img_VRE3 = get_timeseries_image_paths(tile_x, tile_y, "B07")
    image_path_dict_VRE3 = get_image_path_dict(band_img_VRE3)
    vre3_dict = get_tci_dict(image_path_dict_VRE3, mask_list)
    
    tile_dict = {}

    for i in tqdm(range(0, len(mask_list))):
        # run over 184676 mask pixels 
        """
        color_dict = which_pixel_over_timeseries(i, tci_dict)
        tci_red_list = color_dict["red_list"]    # len = 71
        tci_green_list = color_dict["green_list"]
        tci_blue_list = color_dict["blue_list"]

        # if encounter cloud, then use adjacent color. (the previous image or the next image)
        cloud_list = []
        for j in range(0,71):
            r = tci_red_list[j]
            g = tci_green_list[j]
            b = tci_blue_list[j]
            if is_white_pixel(r,g,b) or is_blue_pixel(r,g,b):
                cloud_list.append(j)

        """
        
#         for key,value in color_dict.items():

        # calculate the ndvi value for that one particular pixel 
        nir_list = which_pixel_over_timeseries_ndvi(i, nir_dict)
        red_list = which_pixel_over_timeseries_ndvi(i, red_dict)
        blue_list = which_pixel_over_timeseries_ndvi(i, blue_dict)
        green_list = which_pixel_over_timeseries_ndvi(i, green_dict)
        vre1_list = which_pixel_over_timeseries_ndvi(i, vre1_dict)
        vre2_list = which_pixel_over_timeseries_ndvi(i, vre2_dict)
        vre3_list = which_pixel_over_timeseries_ndvi(i, vre3_dict)
        
        """
        for k in cloud_list:
            if k == 0:
                kk = k + 1
                while kk in cloud_list:
                    kk += 1
                    if kk == 71:
                        break
#                 value[k] = value[kk]
                nir_list[k] = nir_list[kk]
                red_list[k] = red_list[kk]
                blue_list[k] = blue_list[kk]
                green_list[k] = green_list[kk]
                vre1_list[k] = vre1_list[kk]
                vre2_list[k] = vre2_list[kk]
                vre3_list[k] = vre3_list[kk]
            else:
                kk = k - 1
                while kk in cloud_list:
                    kk -= 1
                    if kk == 0:
                        break
#                 value[k] = value[kk]
                nir_list[k] = nir_list[kk]
                red_list[k] = red_list[kk]
                blue_list[k] = blue_list[kk]
                green_list[k] = green_list[kk]
                vre1_list[k] = vre1_list[kk]
                vre2_list[k] = vre2_list[kk]
                vre3_list[k] = vre3_list[kk]
        """
        
        tile_dict[i] = { 
            "nir_list" : nir_list,
            "red_list" : red_list,
            "blue_list" : blue_list,
            "green_list" : green_list,
            "vre1_list" : vre1_list,
            "vre2_list" : vre2_list,
            "vre3_list" : vre3_list
        }
    
    dictkey = "x{}-y{}".format(tile_x,tile_y)
    tile_xy_dict[dictkey] = tile_dict
    break


In [16]:
# tile_dict
# timeseries_list[37]  # 2018-05-26

res = []
for k, v in tqdm( tile_dict.items() ):   # a total of 175950 pixels
    l = []
    for kk, vv in v.items():    # a total of 7 faetures
        l.append(vv[37])
    res.append(l)

In [17]:
df1 = pd.DataFrame(res, columns = ["nir", "red", "blue", "green", "vre1", "vre2", "vre3"])
df1.head()

,nir,red,blue,green,vre1,vre2,vre3
0,6146,1594,2512,2254,2318,4556,5816
1,5490,1692,2452,2192,2318,4556,5816
2,5546,1704,2484,2204,2446,4724,5690
3,5658,1694,2494,2218,2446,4724,5690
4,5558,1776,2462,2280,2544,4854,5914


In [18]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175950 entries, 0 to 175949
Data columns (total 7 columns):
nir      175950 non-null int64
red      175950 non-null int64
blue     175950 non-null int64
green    175950 non-null int64
vre1     175950 non-null int64
vre2     175950 non-null int64
vre3     175950 non-null int64
dtypes: int64(7)
memory usage: 9.4 MB


In [21]:


kmeans = KMeans(n_clusters=13, random_state=0, verbose=True, n_jobs=-1, tol=1e-5, max_iter=900)
X = df1.values
kmeans.fit(X)


Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 362160018931.89435
start iteration
done sorting
end inner loop
Iteration 1, inertia 326204502454.5248
start iteration
done sorting
end inner loop
Iteration 2, inertia 318745604133.8712
start iteration
done sorting
end inner loop
Iteration 3, inertia 316257378844.33264
start iteration
done sorting
end inner loop
Iteration 4, inertia 315187893353.081
start iteration
done sorting
end inner loop
Iteration 5, inertia 314607866652.5325
start iteration
done sorting
end inner loop
Iteration 6, inertia 314213198696.60156
start iteration
done sorting
end inner loop
Iteration 7, inertia 313875888672.4602
start iteration
done sorting
end inner loop
Iteration 8, inertia 313562164397.4626
start iteration
done sorting
end inner loop
Iteration 9, inertia 313259682193.61694
start iteration
done sorting
end inner loop
Iteration 10, inertia 312962924563.519
start iteration
done sorting
end inner loop
Iteration 11, in

Iteration 97, inertia 300239141712.2436
start iteration
done sorting
end inner loop
Iteration 98, inertia 300213280438.632
start iteration
done sorting
end inner loop
Iteration 99, inertia 300187518022.7654
start iteration
done sorting
end inner loop
Iteration 100, inertia 300162669416.27246
start iteration
done sorting
end inner loop
Iteration 101, inertia 300136185594.1621
start iteration
done sorting
end inner loop
Iteration 102, inertia 300109496300.09985
start iteration
done sorting
end inner loop
Iteration 103, inertia 300086328056.97406
start iteration
done sorting
end inner loop
Iteration 104, inertia 300061859668.29376
start iteration
done sorting
end inner loop
Iteration 105, inertia 300035652461.6709
start iteration
done sorting
end inner loop
Iteration 106, inertia 300010281498.64746
start iteration
done sorting
end inner loop
Iteration 107, inertia 299985038456.1656
start iteration
done sorting
end inner loop
Iteration 108, inertia 299960827625.862
start iteration
done sor

Iteration 56, inertia 300749241348.40625
start iteration
done sorting
end inner loop
Iteration 57, inertia 300714720525.0596
start iteration
done sorting
end inner loop
Iteration 58, inertia 300682199004.42694
start iteration
done sorting
end inner loop
Iteration 59, inertia 300648075745.71533
start iteration
done sorting
end inner loop
Iteration 60, inertia 300616866274.26544
start iteration
done sorting
end inner loop
Iteration 61, inertia 300591521919.9992
start iteration
done sorting
end inner loop
Iteration 62, inertia 300570721697.2378
start iteration
done sorting
end inner loop
Iteration 63, inertia 300548646433.02045
start iteration
done sorting
end inner loop
Iteration 64, inertia 300528106717.44653
start iteration
done sorting
end inner loop
Iteration 65, inertia 300510196932.1175
start iteration
done sorting
end inner loop
Iteration 66, inertia 300493794464.64984
start iteration
done sorting
end inner loop
Iteration 67, inertia 300478235271.4214
start iteration
done sorting


Iteration 6, inertia 305522362426.7766
start iteration
done sorting
end inner loop
Iteration 7, inertia 305223433139.31744
start iteration
done sorting
end inner loop
Iteration 8, inertia 304961430626.1706
start iteration
done sorting
end inner loop
Iteration 9, inertia 304745037889.60724
start iteration
done sorting
end inner loop
Iteration 10, inertia 304553388063.47034
start iteration
done sorting
end inner loop
Iteration 11, inertia 304389654647.5528
start iteration
done sorting
end inner loop
Iteration 12, inertia 304255795361.83203
start iteration
done sorting
end inner loop
Iteration 13, inertia 304150934309.8289
start iteration
done sorting
end inner loop
Iteration 14, inertia 304064037477.4869
start iteration
done sorting
end inner loop
Iteration 15, inertia 303988379287.55566
start iteration
done sorting
end inner loop
Iteration 16, inertia 303926539128.297
start iteration
done sorting
end inner loop
Iteration 17, inertia 303877977110.76166
start iteration
done sorting
end in

Iteration 62, inertia 300812596391.78094
start iteration
done sorting
end inner loop
Iteration 63, inertia 300766498242.37415
start iteration
done sorting
end inner loop
Iteration 64, inertia 300722970308.90796
start iteration
done sorting
end inner loop
Iteration 65, inertia 300679139886.9696
start iteration
done sorting
end inner loop
Iteration 66, inertia 300638317516.76465
start iteration
done sorting
end inner loop
Iteration 67, inertia 300598336930.8198
start iteration
done sorting
end inner loop
Iteration 68, inertia 300562470468.6823
start iteration
done sorting
end inner loop
Iteration 69, inertia 300528871178.7381
start iteration
done sorting
end inner loop
Iteration 70, inertia 300493312076.5131
start iteration
done sorting
end inner loop
Iteration 71, inertia 300455248813.71716
start iteration
done sorting
end inner loop
Iteration 72, inertia 300421162064.256
start iteration
done sorting
end inner loop
Iteration 73, inertia 300387592542.54266
start iteration
done sorting
en

...

Iteration 57, inertia 299834726812.9066
start iteration
done sorting
end inner loop
Iteration 58, inertia 299827174219.2406
start iteration
done sorting
end inner loop
Iteration 59, inertia 299819827761.2866
start iteration
done sorting
end inner loop
Iteration 60, inertia 299812712983.1249
start iteration
done sorting
end inner loop
Iteration 61, inertia 299805668284.4597
start iteration
done sorting
end inner loop
Iteration 62, inertia 299798053908.1302
start iteration
done sorting
end inner loop
Iteration 63, inertia 299791437655.8879
start iteration
done sorting
end inner loop
Iteration 64, inertia 299785978902.67706
start iteration
done sorting
end inner loop
Iteration 65, inertia 299780693353.5741
start iteration
done sorting
end inner loop
Iteration 66, inertia 299775022722.238
start iteration
done sorting
end inner loop
Iteration 67, inertia 299768908360.8327
start iteration
done sorting
end inner loop
Iteration 68, inertia 299763900572.3153
start iteration
done sorting
end inn

Iteration 72, inertia 312254037317.31146
start iteration
done sorting
end inner loop
Iteration 73, inertia 312248235988.8498
start iteration
done sorting
end inner loop
Iteration 74, inertia 312242111524.74316
start iteration
done sorting
end inner loop
Iteration 75, inertia 312236867673.3284
start iteration
done sorting
end inner loop
Iteration 76, inertia 312231965836.25745
start iteration
done sorting
end inner loop
Iteration 77, inertia 312226685307.7068
start iteration
done sorting
end inner loop
Iteration 78, inertia 312221644986.13403
start iteration
done sorting
end inner loop
Iteration 79, inertia 312217129755.8081
start iteration
done sorting
end inner loop
Iteration 80, inertia 312213338634.8518
start iteration
done sorting
end inner loop
Iteration 81, inertia 312209793022.34033
start iteration
done sorting
end inner loop
Iteration 82, inertia 312206353029.7828
start iteration
done sorting
end inner loop
Iteration 83, inertia 312202953235.2125
start iteration
done sorting
en

Iteration 174, inertia 299801340586.9785
start iteration
done sorting
end inner loop
Iteration 175, inertia 299797017764.32513
start iteration
done sorting
end inner loop
Iteration 176, inertia 299792760757.4513
start iteration
done sorting
end inner loop
Iteration 177, inertia 299788895177.20605
start iteration
done sorting
end inner loop
Iteration 178, inertia 299785172125.49896
start iteration
done sorting
end inner loop
Iteration 179, inertia 299781243864.9708
start iteration
done sorting
end inner loop
Iteration 180, inertia 299777179128.2549
start iteration
done sorting
end inner loop
Iteration 181, inertia 299773480977.3829
start iteration
done sorting
end inner loop
Iteration 182, inertia 299769598884.00287
start iteration
done sorting
end inner loop
Iteration 183, inertia 299765434272.84436
start iteration
done sorting
end inner loop
Iteration 184, inertia 299761467960.3887
start iteration
done sorting
end inner loop
Iteration 185, inertia 299757339690.612
start iteration
done

Iteration 55, inertia 300718737341.9288
start iteration
done sorting
end inner loop
Iteration 56, inertia 300675940650.458
start iteration
done sorting
end inner loop
Iteration 57, inertia 300635177223.5137
start iteration
done sorting
end inner loop
Iteration 58, inertia 300595563495.9521
start iteration
done sorting
end inner loop
Iteration 59, inertia 300558988698.2808
start iteration
done sorting
end inner loop
Iteration 60, inertia 300524925174.9047
start iteration
done sorting
end inner loop
Iteration 61, inertia 300488385744.0705
start iteration
done sorting
end inner loop
Iteration 62, inertia 300450861832.3274
start iteration
done sorting
end inner loop
Iteration 63, inertia 300416265970.48865
start iteration
done sorting
end inner loop
Iteration 64, inertia 300384354977.8804
start iteration
done sorting
end inner loop
Iteration 65, inertia 300352639095.3382
start iteration
done sorting
end inner loop
Iteration 66, inertia 300321193515.43506
start iteration
done sorting
end in

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=900,
    n_clusters=13, n_init=10, n_jobs=-1, precompute_distances='auto',
    random_state=0, tol=1e-05, verbose=True)

In [1]:
# len(mask_list)
type_y = kmeans.labels_
len(type_y)

NameError: name 'kmeans' is not defined

In [27]:
mask_list[0]

(0, 0)

In [28]:
d1 = {}
cnt = 0
for ml in tqdm(mask_list):
    x = ml[0]
    y = ml[1]
    d1["x{}-y{}".format(x,y)] = type_y[cnt]
    cnt+=1

In [32]:
xy_pair[0]

('6144', '5120')

In [33]:
# draw color change over timeseries

color_type = {
    "0": (255,0,0),
    "1": (255,255,0),
    "2": (255,0,255),
    "3": (0,0,255),
    "4": (0,255,255),
    "5": (0,255,0),
    "6": (255,145,0),
    "7": (255,255,255),
    "8": (150,150,150),
    "9": (111,1,1),
    "10": (74,1,111),
    "11": (229,204,255),
    "12": (255,229,204)
}

data = np.zeros( (512,512,3), dtype=np.uint8 )

# data[512,512] = (254,0,0)       # Makes the middle pixel red
# data[512,513] = (0,0,255)       # Makes the next pixel blue

for x in range(0,512):
    for y in range(0,512):
            k = "x{}-y{}".format(x,y)
            if k in d1:
                s_type = d1[k]
                data[y,x] = color_type[str(s_type)]
            else:
                data[y,x] = (0, 0, 0)

img = smp.toimage( data )
img.show()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
